In [1]:
pip install ipydeps


     |████████████████████████████████| 61kB 3.8MB/s 
     |████████████████████████████████| 2.3MB 10.3MB/s 
  Created wheel for ipydeps: filename=ipydeps-0.10.0-cp36-none-any.whl size=10798 sha256=b5da40ca92811791852240011197c7edec548a152c4492c78a5110fb94f1e639
  Stored in directory: /root/.cache/pip/wheels/e3/4a/6f/2a1810efd60598edd3f3a3d800bb46f734407dd62c18710534
  Created wheel for pypki2: filename=pypki2-0.11.0-cp36-none-any.whl size=10428 sha256=9c4e1bc83994fac920ad330e77a2745d8082574c1b1d192bb75e084e70a045c0
  Stored in directory: /root/.cache/pip/wheels/94/40/0b/348ab8c4a7ccd1f846f0b7a011065f70b06a53c7accf84a301
Successfully built ipydeps pypki2


In [0]:
import ipydeps

In [3]:
import tensorflow 
tensorflow.keras.__version__

'2.2.4-tf'

In [4]:
import keras, random
keras.__version__
import os
random.seed(1)
from google.colab import drive
drive.mount('/gdrive', force_remount=True)
base_dir='/gdrive/My Drive/Project'

Using TensorFlow backend.


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [0]:
#datasets
taiwan = os.path.join(base_dir, 'taiwan.txt')
ch = os.path.join(base_dir, 'ch.txt')

In [0]:
#organize taiwanese news dataset into list of articles
t_file = open(taiwan).readlines()
t_list = [line.rstrip('\n') for line in t_file]
t_no_newline = "".join(t_list)
t_list = t_no_newline.split("------------------------------------------------------------------------------------------------------------------------------------------------------")

#organize chinese news dataset into list of articles
c_file = open(ch).readlines()
c_list = [line.rstrip('\n') for line in c_file]
c_no_newline = "".join(c_list)
c_list = c_no_newline.split("------------------------------------------------------------------------------------------------------------------------------------------------------")

In [7]:
import pandas as pd
#taiwanese = 1
df = pd.DataFrame({"Articles": t_list,"Classification": 1})
print(len(df))

#chinese = 0
df2 = pd.DataFrame({"Articles": c_list,"Classification": 0})
print(len(df2))

data = df.append(df2)
data = data.sample(frac=1)
data

NameError: ignored

In [0]:
import re
#clean text by only including chinese characters

#taiwanese dataset
taiwan_list = []
for article in t_list:
  single_article = []
  for j in re.findall(r'[\u4e00-\u9fff]+', article):
    single_article.append(j)
  taiwan_list.append("".join(single_article))

#chinese dataset
china_list = []
for article in c_list:
  single_article = []
  for j in re.findall(r'[\u4e00-\u9fff]+', article):
    single_article.append(j)
  china_list.append("".join(single_article))
#https://stackoverflow.com/questions/2718196/find-all-chinese-text-in-a-string-using-python-and-regex

In [9]:
import os       #importing os to set environment variable
def install_java():
  !apt-get install -y openjdk-8-jdk-headless -qq > /dev/null      #install openjdk
  os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"     #set environment variable
  !java -version       #check java version
install_java()

#https://stackoverflow.com/questions/51287258/how-can-i-use-java-in-google-colab

openjdk version "11.0.6" 2020-01-14
OpenJDK Runtime Environment (build 11.0.6+10-post-Ubuntu-1ubuntu118.04.1)
OpenJDK 64-Bit Server VM (build 11.0.6+10-post-Ubuntu-1ubuntu118.04.1, mixed mode, sharing)


In [10]:
ipydeps.pip("hanziconv")

In [0]:
#convert all text in Chinese articles into traditional characters
from hanziconv import HanziConv
tr_china_list = []
for article in china_list:
  tr_china_list.append(HanziConv.toTraditional(article))
#https://pypi.org/project/hanziconv/

In [12]:
ipydeps.pip("jieba")

In [13]:
import jieba
#segment text into words with spaces as delimiters

#taiwanese dataset
t_segmented_list = []
for article in taiwan_list:
  seg_list = jieba.cut(article)
  t_segmented_list.append(' '.join(list(seg_list)))

#chinese dataset
c_segmented_list = []
for article in tr_china_list:
  seg_list = jieba.cut(article)
  c_segmented_list.append(' '.join(list(seg_list)))

#https://github.com/fxsjy/jieba

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.785 seconds.
Prefix dict has been built successfully.


In [0]:
#taiwanese cleaned text document
taiwan_clean = open("taiwan_clean.txt", "w")
for article in t_segmented_list:
  taiwan_clean.write(article + "\n")
taiwan_clean.close()

#chinese cleaned text document
china_clean = open("china_clean.txt", "w")
for article in c_segmented_list:
  china_clean.write(article + "\n")
china_clean.close()


In [0]:
#cleaned datasets
taiwan_clean = os.path.join(base_dir, 'taiwan_clean.txt')
china_clean = os.path.join(base_dir, 'china_clean.txt')

In [6]:
import pandas as pd
t_clean_data = pd.read_csv(taiwan_clean, header = None, names = ["Articles"])
t_clean_data["Classification"] = 1

c_clean_data = pd.read_csv(china_clean, header = None, names = ["Articles"])
c_clean_data["Classification"] = 0

data = t_clean_data.append(c_clean_data)
data = data.sample(frac=1)
data

,Articles,Classification
36921,驚捷 運上 遇到 仙人 道長 網友 難道 是 穿越 生活 自由 時報 電子 報即 時 政治 ...,1
4445,中新 網視 頻雲南景 榖 地震 獲 提前 預警 災區 采集 平 颱 首次 啓 用 首頁 滾動...,0
3619,節能 減碳 梧棲 元宵 燈節 踩 街 暖 身發 送個 燈籠 生活 自由 時報 電子 報為 達...,1
14607,港媒 不容 黑手 伸進 校園 教育局 和 校方 須負 起責任 中 新 網首頁 滾動國 內國際...,0
4975,川普 使出 關稅 殺 手鐧 中國 今年 成長 率 難保 自由 財經 即 時 政治 社會 生活...,1
...,...,...
20581,小稅俠 帶 你 看 減稅 降費首 發進 校園 中國 新聞網 湖南 當前 位置 首頁 新聞 內...,0
60153,集齊頂 尖造車 技術 的 恒大 在 下 一盤 什麼 棋中國 新聞網 山西 影響 山西 的 力...,0
61325,柬埔寨 奧委員會 主席 全球 徵召 柬埔寨 運動員備 戰東 南亞 運動會 中 新 網廣西 新...,0
3393,山東 實施 流程 再造 推進 一窗 受理 一次 辦好 首頁 頻道 正文 山東 實施 流程 再...,0


In [78]:
 ipydeps.pip("fasttext")

In [0]:
#fasttext.util.download_model('zh', if_exists='ignore') 
#ft = fasttext.load_model('cc.en.300.bin')

In [0]:
from keras.preprocessing.text import one_hot

In [0]:
#one hot encode

#taiwan
t_clean_file = open(taiwan_clean).readlines()
t_clean_oh = []
for article in t_clean_file:
  t_clean_oh.append(one_hot(article,1000000))

#china
c_clean_file = open(china_clean).readlines()
c_clean_oh = []
for article in c_clean_file:
  c_clean_oh.append(one_hot(article,1000000))